## Example of model with sliders & wrapping functions

In [ ]:
import underworld as uw
import glucifer
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output

Setup parameters
-----

In [ ]:
outputPath = './SteadyState/'
# Set box size.
boxHeight = 1.0
boxLength = 2.0
# Set the resolution.
resx = 16
resy = widgets.IntSlider(description='no. elements', min=1,max=100, value=8)
heating = widgets.IntSlider(description='heating', min=-10,max=10, value=2)
outputbox = widgets.Text(
    value='output/steadystate',
    description='output path:',
    disabled=False
)

In [ ]:
xx = widgets.Button(
    description='Do science',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)

In [ ]:
def display_options():
    display(resy)
    display(heating)
    display(xx)
    display(outputbox)

Create mesh and variables
------

In [ ]:
def make_model(d):
    global outputPath 
    outputPath = str(outputbox.value)
    import os
    outputPath = os.path.abspath(outputPath)+'/'
    # build output path on root proc
    if uw.rank() == 0:
        if not os.path.exists(outputPath):
            os.makedirs(outputPath)

    mesh = uw.mesh.FeMesh_Cartesian( elementType = ("Q1/dQ0"), 
                                 elementRes  = (resy.value, resy.value), 
                                 minCoord    = (0., 0.), 
                                 maxCoord    = (boxLength, boxHeight))
    # Create mesh variables for the temperature field & initialise. 
    temperatureField = mesh.add_variable( nodeDofCount=1 )
    temperatureField.data[:] = 0.

    # visualisation of temperature field & mesh
    store = glucifer.Store(outputPath+'/glucifer.gldb')
    fig = glucifer.Figure( store, figsize=(800,400) )
    fig.append(glucifer.objects.Mesh( mesh ))
    fig.Surface( mesh, temperatureField, colours="blue white red", name="Temperature" )
    fig.save()
    store.step = store.step + 1
    
    jWalls = mesh.specialSets["MinJ_VertexSet"] + mesh.specialSets["MaxJ_VertexSet"]
    tempBC = uw.conditions.DirichletCondition( variable=temperatureField, indexSetsPerDof=(jWalls,) )
    # set bottom wall temperature bc
    for index in mesh.specialSets["MinJ_VertexSet"]:
        temperatureField.data[index] = 1.0
    # set top wall temperature bc
    for index in mesh.specialSets["MaxJ_VertexSet"]:
        temperatureField.data[index] = 0
    
    heatequation = uw.systems.SteadyStateHeat(temperatureField = temperatureField, 
                                          fn_heating=heating.value ,
                                          fn_diffusivity = 1.0, 
                                          conditions = tempBC)
    # get the default heat equation solver
    heatsolver = uw.systems.Solver(heatequation)
    # solve.solve()
    heatsolver.solve()
    fig.save()
    return temperatureField

In [ ]:
def vizualise(path=None):
    # if no path provided use the global value from the widget
    if path == None:
        global outputPath
        path = outputPath
    import os
    if not os.path.exists(path):
        print("Can't find ", path)
        return

    vis = glucifer.lavavu.Viewer(database=path+'/glucifer.gldb')
    vis.control.Panel()

    vis.control.TimeStepper()
    vis.control.ObjectList()
    vis.control.show()

In [ ]:
xx.on_click(make_model)

In [ ]:
display_options()

In [ ]:
# vizualise()